Click the button below if you want to see the code behind the widgets. If you do click  and want to go back to the 'no-code' view, scroll down until you find the 'hide code' button.

In [1]:
"""
MIT License

Copyright (c) 2020 Sylvain Barde - University of Kent

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rcParams
from IPython.display import display, HTML

javascript_functions = {False: "hide()", True: "show()"}
button_descriptions  = {False: "Click to show code", True: "Click to hide code"}

def toggle_code(state):
    """
    Toggles the JavaScript show()/hide() function on the div.input element.
    """

    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_functions[state],)
    output        = output_string.format(*output_args)

    display(HTML(output))
    
def button_action(value):
    """
    Calls the toggle_code function and updates the button description.
    """
    state = value.new

    toggle_code(state)

    value.owner.description = button_descriptions[state]

def Elasticities_widget(Qmax_init = 15, Pmax_init = 30, Qval_init = 5, a_d_init = 2,
    b_d_init = 25, a_s_init = 2, b_s_init = -3, Dflag_init = True, Sflag_init = False):
        
    # Declare widgets for interactive input
    Qmax_slider = widgets.IntSlider(min=5,
                                 max=1000,
                                 step=1,
                                 description=r'Maximum $Q$:',
                                 value = Qmax_init,
                                 continuous_update =False)
    Pmax_slider = widgets.IntSlider(min=5,
                                 max=1000,
                                 step=1,
                                 description=r'Maximum $P$:',
                                 value = Pmax_init,
                                 continuous_update =False)
    Qval_slider = widgets.FloatSlider(min=0.001,
                                 max=15,
                                 description='Pick a Quantity:',
                                 value = Qval_init,
                                 continuous_update =False)
    a_d_slider = widgets.FloatSlider(min=0.001,
                                 max=100,
                                 description= r'$a_d$:',
                                 value = a_d_init,
                                 continuous_update =False)
    b_d_slider = widgets.FloatSlider(min=0,
                                 max=1000,
                                 description= r'$b_d$:',
                                 value=b_d_init,
                                 continuous_update =False)
    a_s_slider = widgets.FloatSlider(min=0.001,
                                 max=100,
                                 description= r'$a_s$:',
                                 value = a_s_init,
                                 continuous_update =False)
    b_s_slider = widgets.FloatSlider(min=-1000,
                                 max=1000,
                                 description= r'$b_s$:',
                                 value = b_s_init,
                                 continuous_update =False)
    Dflag_check = widgets.Checkbox(value = Dflag_init,
                                   description='Include Demand',
                                   disabled=False,
                                   indent=True) 
    Sflag_check = widgets.Checkbox(value = Sflag_init,
                                   description='Include Supply',
                                   disabled=False,
                                   indent=True)
    
    # Link widgets as required
    widgets.jslink((Qmax_slider,'value'),(Qval_slider,'max'))
    widgets.jslink((b_d_slider,'value'),(b_s_slider,'max'))

    def elasticity_plot(Qmax, Pmax, Qval, a_d, b_d, a_s, b_s, Dflag, Sflag):

        # create a quantity vector, calculate supply/demand vectors
        Q = np.arange(0,Qmax)
        P_s = a_s*Q + b_s
        P_d = -a_d*Q + b_d

        # Calculate equilibrium quantity/price
        Qeq = (b_d-b_s)/(a_s + a_d)
        Peq = a_s*Qeq + b_s

        # Calculate prices for selected value
        Pval_s = a_s*Qval + b_s
        Pval_d = -a_d*Qval + b_d

        # Create figure, plot supply/demand
        fig, ax = plt.subplots(figsize=(20,10))
        if Sflag is True:
            ax.plot(Q, P_s,'r', linewidth=2, alpha=0.6,
                    label=r'Inverse Supply $\quad P_s = a_s Q + b_s$')
        if Dflag is True:
            ax.plot(Q, P_d,'b', linewidth=2, alpha=0.6,
                    label=r'Inverse Demand $\quad P_d = -a_d Q + b_d$')

        # Add markers for the price/quantity points, with dotted lines
        mrkrSize = 2*rcParams['lines.markersize'] ** 2
        if Sflag is True and Dflag is True:
            ax.scatter(Qeq, Peq, s=mrkrSize, c='k', label='Equilibrium')       
            ax.scatter(Qval, Pval_s, s=mrkrSize, c='k', alpha=0.6, 
                       label='Selection')
            ax.scatter(Qval, Pval_d, s=mrkrSize, c='k', alpha=0.6)

            ax.plot([Qeq,Qeq],[0,Peq],'k--',linewidth=1)
            ax.plot([0,Qeq],[Peq,Peq],'k--',linewidth=1)
            ax.plot([0,Qval],[Pval_d,Pval_d],'k--',linewidth=1)
            ax.plot([Qval,Qval],[0,max(Pval_s,Pval_d)],'k--',linewidth=1)
            ax.plot([0,Qval],[Pval_s,Pval_s],'k--',linewidth=1)
            ax.annotate(r'$Q^*$',[Qeq,0], xytext = [Qeq+0.15,0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)
            ax.annotate(r'$P^*$',[0,Peq], xytext = [0.15,Peq+0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)
            ax.annotate(r'$Q$',[Qval,0], xytext = [Qval+0.15,0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)
            ax.annotate(r'$P_s$',[0,Pval_s], xytext = [0.15,Pval_s+0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)
            ax.annotate(r'$P_d$',[0,Pval_d], xytext = [0.15,Pval_d+0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)

        elif Sflag is True and Dflag is False:
            ax.scatter(Qval, Pval_s, s=mrkrSize, c='k', alpha=0.6, 
                       label='Selection')
            ax.plot([0,Qval],[Pval_s,Pval_s],'k--',linewidth=1)
            ax.plot([Qval,Qval],[0,Pval_s],'k--',linewidth=1)    
            ax.annotate(r'$Q$',[Qval,0], xytext = [Qval+0.15,0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)
            ax.annotate(r'$P_s$',[0,Pval_s], xytext = [0.15,Pval_s+0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)

        elif Sflag is False and Dflag is True:
            ax.scatter(Qval, Pval_d, s=mrkrSize, c='k', alpha=0.6, 
                       label='Selection')
            ax.plot([0,Qval],[Pval_d,Pval_d],'k--',linewidth=1)
            ax.plot([Qval,Qval],[0,Pval_d],'k--',linewidth=1)
            ax.annotate(r'$Q$',[Qval,0], xytext = [Qval+0.15,0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)
            ax.annotate(r'$P_d$',[0,Pval_d], xytext = [0.15,Pval_d+0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)

        # Add elasticity annotations
        if Sflag is True:
            EsStr = r'$E_p^S = {:.2f}$'.format(Pval_s/(Qval*a_s))
            ax.annotate(EsStr,[Qval,Pval_s],
                        xytext = [Qval+0.25,Pval_s-1], 
                        xycoords ='data',
                        fontsize = 25,
                        clip_on = True)
        if Dflag is True:
            EdStr = r'$E_p^D = {:.2f}$'.format(-Pval_d/(Qval*a_d))
            ax.annotate(EdStr,[Qval,Pval_d],
                        xytext = [Qval+0.25,Pval_d],
                        xycoords ='data',
                        fontsize = 25,
                        clip_on = True)

        # Add legend and format axes to look nice
        if Sflag is True or Dflag is True:
            ax.legend(loc='upper center', frameon=False,prop={'size':20})
        ax.autoscale(enable=True, axis='both', tight=True)
        ax.set_ylim(top = Pmax, bottom = 0)
        ax.set_xlim(right = Qmax, left = 0)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.set_xlabel(r'$Q$', fontdict = {'fontsize': 25},position=(1, 0))
        ax.set_ylabel(r'$P$', fontdict = {'fontsize': 25},position=(0, 1), rotation=0)
        ax.plot(1, 0, ">k", transform=ax.get_yaxis_transform(), clip_on=False)
        ax.plot(0, 1, "^k", transform=ax.get_xaxis_transform(), clip_on=False)
        plt.tick_params(labelsize=20)
        plt.tight_layout()
    
    out = widgets.interactive_output(elasticity_plot, {'Qmax': Qmax_slider,
                                                       'Pmax': Pmax_slider,
                                                       'Qval': Qval_slider, 
                                                       'a_d': a_d_slider,
                                                       'b_d': b_d_slider,
                                                       'a_s': a_s_slider, 
                                                       'b_s': b_s_slider, 
                                                       'Dflag': Dflag_check,
                                                       'Sflag': Sflag_check})

    output = widgets.VBox([out,
                  widgets.HBox([Qval_slider,
                               Qmax_slider,
                               Pmax_slider]),
                  widgets.HBox([Dflag_check, 
                                a_d_slider, 
                                b_d_slider]),
                  widgets.HBox([Sflag_check, 
                                a_s_slider, 
                                b_s_slider])])
    display(output)

def Slope_widget(Qmax_init = 15, Pmax_init = 10, Qeq_init = 7.5, Peq_init = 5,
    E_s_init = 2.5, E_d_init = -1.5, Dflag_init = True, Sflag_init = True):
        
    # Declare widgets for interactive input
    Qmax_slider = widgets.IntSlider(min=5,
                                 max=1000,
                                 step=1,
                                 description=r'Maximum $Q$:',
                                 value = Qmax_init,
                                 continuous_update =False)
    Pmax_slider = widgets.IntSlider(min=5,
                                 max=1000,
                                 step=1,
                                 description=r'Maximum $P$:',
                                 value = Pmax_init,
                                 continuous_update =False)
    Qeq_slider = widgets.FloatSlider(min=0.001,
                                 max=15,
                                 description='Pick a Quantity:',
                                 value = Qeq_init,
                                 continuous_update =False)
    Peq_slider = widgets.FloatSlider(min=0.001,
                                 max=200,
                                 description= 'Pick a Price:',
                                 value = Peq_init,
                                 continuous_update =False)
    E_d_slider = widgets.FloatSlider(min=-15,
                                 max=-0.001,
                                 description= 'Price elasticity of demand:',
                                 value=E_d_init,
                                 continuous_update =False)
    E_s_slider = widgets.FloatSlider(min=0.001,
                                 max=15,
                                 description= 'Price elasticity of supply:',
                                 value = E_s_init,
                                 continuous_update =False)
    Dflag_check = widgets.Checkbox(value = Dflag_init,
                                   description='Include Demand',
                                   disabled=False,
                                   indent=True) 
    Sflag_check = widgets.Checkbox(value = Sflag_init,
                                   description='Include Supply',
                                   disabled=False,
                                   indent=True)
    
    # Link widgets as required
    widgets.jslink((Qmax_slider,'value'),(Qeq_slider,'max'))
    widgets.jslink((Pmax_slider,'value'),(Peq_slider,'max'))

    def slope_plot(Qmax, Pmax, Qeq, Peq , E_d, E_s, Dflag, Sflag):

        # Calculate demand supply curve parameters from input    
        a_s = E_s*Peq/Qeq
        b_s = Peq-a_s*Qeq
        a_d = E_d*Peq/Qeq
        b_d = Peq-a_d*Qeq

        # create a quantity vector, calculate supply/demand vectors
        Q = np.asarray([Qeq-0.05*Qmax,Qeq+0.05*Qmax])
        P_s = a_s*Q + b_s
        P_d = a_d*Q + b_d

        # Create figure, plot supply/demand approximations with arrows
        fig, ax = plt.subplots(figsize=(20,10))
        if Sflag is True:
            ax.plot(Q, P_s,'r', linewidth=2, alpha=0.6,
                    label= 'Local approximation of Supply curve')
            ax.annotate('', xy=(Q[0],P_s[0]), xytext=(Q[0]-0.01,P_s[0]-0.01*a_s),
                        arrowprops={'arrowstyle': '<-', 
                                    'lw': 2, 
                                    'color': 'r', 
                                    'alpha':0.6})
            ax.annotate('', xy=(Q[1],P_s[1]), xytext=(Q[1]+0.01,P_s[1]+0.01*a_s),
                        arrowprops={'arrowstyle': '<-', 
                                    'lw': 2, 
                                    'color': 'r', 
                                    'alpha':0.6})

        if Dflag is True:
            ax.plot(Q, P_d,'b', linewidth=2, alpha=0.6,
                    label= 'Local approximation of Demand curve')
            ax.annotate('', xy=(Q[0],P_d[0]), xytext=(Q[0]-0.01,P_d[0]-0.01*a_d),
                        arrowprops={'arrowstyle': '<-', 
                                    'lw': 2, 
                                    'color': 'b', 
                                    'alpha':0.6})
            ax.annotate('', xy=(Q[1],P_d[1]), xytext=(Q[1]+0.01,P_d[1]+0.01*a_d),
                        arrowprops={'arrowstyle': '<-', 
                                    'lw': 2, 
                                    'color': 'b', 
                                    'alpha':0.6})

        # Add markers for the price/quantity points, with dotted lines
        mrkrSize = 2*rcParams['lines.markersize'] ** 2
        if Sflag is True and Dflag is True:
            ax.scatter(Qeq, Peq, s=mrkrSize, c='k', label='Equilibrium')       

            ax.plot([Qeq,Qeq],[0,Peq],'k--',linewidth=1)
            ax.plot([0,Qeq],[Peq,Peq],'k--',linewidth=1)
            ax.annotate(r'$Q^*={:.2f}$'.format(Qeq),[Qeq,0], xytext = [Qeq+0.15,0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)
            ax.annotate(r'$P^*={:.2f}$'.format(Peq),[0,Peq], xytext = [0.15,Peq+0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)

    #    # Add elasticity annotations
        if Sflag is True:
            EsStr = r'$E_p^S = {:.2f}$'.format(E_s)
            ax.annotate(EsStr,[Q[1],P_s[1]],
                        xytext = [Q[1]+0.25,P_s[1]], 
                        xycoords ='data',
                        fontsize = 25,
                        clip_on = True)
        if Dflag is True:
            EdStr = r'$E_p^D = {:.2f}$'.format(E_d)
            ax.annotate(EdStr,[Q[1],P_d[1]],
                        xytext = [Q[1]+0.25,P_d[1]-0.5], 
                        xycoords ='data',
                        fontsize = 25,
                        clip_on = True)

        # Add legend and format axes to look nice
        if Sflag is True or Dflag is True:
            ax.legend(loc='upper center', frameon=False,prop={'size':20})
        ax.autoscale(enable=True, axis='both', tight=True)
        ax.set_ylim(top = Pmax, bottom = 0)
        ax.set_xlim(right = Qmax, left = 0)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.set_xlabel(r'$Q$', fontdict = {'fontsize': 25},position=(1, 0))
        ax.set_ylabel(r'$P$', fontdict = {'fontsize': 25},position=(0, 1), rotation=0)
        ax.plot(1, 0, ">k", transform=ax.get_yaxis_transform(), clip_on=False)
        ax.plot(0, 1, "^k", transform=ax.get_xaxis_transform(), clip_on=False)
        plt.tick_params(labelsize=20)
        plt.tight_layout()
    
    out = widgets.interactive_output(slope_plot, {'Qmax': Qmax_slider,
                                                   'Pmax': Pmax_slider,
                                                   'Qeq': Qeq_slider, 
                                                   'Peq': Peq_slider,
                                                   'E_d': E_d_slider,
                                                   'E_s': E_s_slider, 
                                                   'Dflag': Dflag_check,
                                                   'Sflag': Sflag_check})

    output = widgets.VBox([out,
                  widgets.HBox([Qmax_slider,
                               Pmax_slider,
                               Qeq_slider,
                               Peq_slider]),
                  widgets.HBox([Dflag_check, 
                                E_d_slider, 
                                Sflag_check,
                                E_s_slider])])
    display(output)    
    

In [2]:
state = False
toggle_code(state)

button = widgets.ToggleButton(state, description = button_descriptions[state])
button.observe(button_action, "value")

display(button)

ToggleButton(value=False, description='Click to show code')

### Elasticities: An application of linear functions in economics

Elasticities are a really important concept in economics, and we will be referring to them a lot not only in this module, but also throughout your degree. Their definition is rather simple, yet they are incredibly useful. Let's start with definitions. You may have seen this before, ad you'll definitely see it again, but I want to make sure everybody is clear on we are talking about before we start playing around with the concept and providing some illustrations.

Suppose you have two variables $A$ and $B$, the elasticity of $B$ with respect to $A$ is the percentage change in $B$ (written $\%\Delta B$) divided by the percentage change in $A$.

$$E_A^B = \frac{\%\Delta B}{\%\Delta A}$$

Imagine that the elasticity is equal to 0.5. The way we interpret this is to say that if $A$ increases by 1%, B will increase by 0.5%. If $A$ increases by 10%, B will increase by 5%. A common elasticity used in economics is the price elasticity of demand (or more precisely the elasticity of demand with respect to price).

$$E_P^D  = \frac{\%\Delta Q_d}{\%\Delta P}$$

The price elasticity of demand is usually negative (higher prices lead to lower demand) and it tells us how responsive demand is to price. If it is less than one in absolute value, so $|E_P^D| < 1$, a 1% increase in price leads to a less than 1% fall in quantity demanded, and we say that demand in *inelastic*. If instead $|E_P^D| >,  1$ a 1% increase in price leads to a more than 1% fall in quantity demanded, and we say that demand in *elastic*.

### What do linear functions have to do with this?

You may be asking yourselves this, and it's a valid point. To explain the connection, we have to get a more detailed definition of an elasticity. The first step it to look at the idea of percentage change: How do we calculate $\%\Delta Q_d$? Well, the definition of a percentage is:

$$\%\Delta Q_d = 100\times\frac{\Delta Q_d}{Q_d}$$

Here $\Delta Q_d$ is the change in quantity (either an increase or decrease). So if your initial quantity is 500 and it increases to 510, then the change $\Delta Q_d$ is 10. This corresponds to a percentage change of 2%:

$$\%\Delta Q_d = 100\times\frac{10}{500} = 2$$

The same formula applies to the percentage change in price $\%\Delta P. If we replace the percentage change formulas in the definition of an elasticity, by the definitions we just provided we get:

$$E_P^D  = \frac{100\times\frac{\Delta Q_d}{Q_d}}{100\times\frac{\Delta P}{P}}$$

This doesn't look very friendly, but not to worry: it does simplify nicely. First of all, the 100 terms in the numerator and denominator cancel out. Secondly, we can make use of the fact that division is simply multiplication be the inverse (remember, dividing by 3 is the same as multiplying by one third!):

$$E_P^D  = \frac{\frac{\Delta Q_d}{Q_d}}{\frac{\Delta P}{P}}  \quad = \quad \frac{\Delta Q_d}{Q_d}\frac{P}{\Delta P}$$

If we rearrange the denominators, we get the standard definitions of the price elasticity of demand:

$$E_P^D  = \frac{\Delta Q_d}{\Delta P}\frac{P}{Q_d}$$

The first fraction in this definition is simply the slope of the demand curve $\frac{\Delta Q_d}{\Delta P}$. For a linear function, this is always fixed. The second fraction is simply the price/quantity ratio for a given point on the demand curve. an important implication is that **very point on the demand curve will have a different elasticity**. This can easily be illustrated: change the quantity in the diagram below, and you will see the elasticity change.


In [3]:
Elasticities_widget()

### Going off on a tangent: 'elastic' and 'inelastic' market curves

This is a bit of tangent, but it does offer an important clarification of how economists use the terms 'elastic' or 'inelastic' for supply and demand curves on market diagrams. Before jumping in, you need to know that you can define price elasticities of supply $E_P^S$ just like the price elasticity of demand seen above. The only difference is that we look at how the quantity supplied $Q^s$ changes as price changes, rather than quantity demanded $Q^d$.

OK, so you will sometimes be shown a diagram where either the supply curve, or the demand curve, or both, have a very steep slope (see below), and the curves will be referred to as *inelastic*. This is broadly correct, however, it is a bit of a shortcut, and it gives the impression that the fixed slope corresponds to a fixed elasticity. If you check the example below, which shows an 'inelastic' market, you will see that the elasticities *still change* on the curves when you change the quantities. So while it is true that the elasticities you will see are all below 1 in magnitude, (hence the term 'inelastic'), it would be wrong to conclude that they are constant.

In [4]:
Elasticities_widget(Qmax_init = 15, Pmax_init = 30, Qval_init = 3.3, a_d_init = 75,
    b_d_init = 275, a_s_init = 50, b_s_init = -150, Dflag_init = True, Sflag_init = True)

A similar argument can be made when you have *elastic* demand and supply: the demand or supply curves will have slopes that are very flat, but again, if you look at the example below you will see that the elasticity still changes as you travel up/down the curves.

So, just to summarise: **constant slope does not mean constant elasticity**. Just because the concepts are related does not mean they are the same.!!

In [5]:
Elasticities_widget(Qmax_init = 15, Pmax_init = 30, Qval_init = 6, a_d_init = 0.5,
    b_d_init = 22, a_s_init = 0.75, b_s_init = 10, Dflag_init = True, Sflag_init = True)

### Back to the main event: why are elasticities useful?

Now that we know that the elasticity of a demand or supply curve at a given point is the product of the slope and the price/quantity ration at that point, we may well ask: why is that useful? Well, in practice economists tend to use elasticities in the opposite direction to what they were taught.

When we learn/teach elasticities, we give you the price, quantity and the slope of the demand/supply curves, and use that information to calculate elasticities. This is an important step, because it helps students understand the concept of an elasticity. In practice however, it is important to understand that *no economist has ever seen a supply or a demand curve*!! Demand and supply are very helpful theoretical constructs that help us model what is happening in the economy, but nobody has ever seen one on the high street or in a supermarket... 

So when an economist wants to give some policy advice about a specific market, they initially have **no idea** what the demand/supply curves look like. They can observe the market price and the quantity being exchanged, but that's it: they can't see the curves... 

What they can do is collect data on prices and quantities over time on that market, and use statistical techniques to get estimates of the elasticities (how sensitive the quantities are to price changes). This calculating elasticities from data is something you will see next year in econometrics. Then, once they know the elasticity, the price and the quantity, they can invert the elasticity formula to calculate the slope of the supply/demand curves around the market equilibrium. 

$$ \frac{\Delta Q_d}{\Delta P} = E_P^D \frac{Q_d}{P}$$

So elasticities, which we can measure from data, **allow us to build a linear approximation of the supply/demand curves look around the market equilibrium**!! This is illustrated below: pick a price and a quantity for your market, and pick some elasticities. The widget will plot what the supply and demand curves look like around your chosen equilibrium. Try changing the quantity or price for a given level of the demand/supply elasticities, and get an idea of how the local demand/supply curves respond...

In [6]:
Slope_widget()

#### Wrapping up...

In conclusion, elasticities are used all the time by professional economists, but perhaps not in the way you think. They are very useful *as is* to give an idea of how one variable might react to another. But they're also very very useful in allowing us to link our models to the data, because they allow us to get the slopes of our functions from data we estimate.

Just because we give you practice exercises in which we ask you to calculate elasticities given a known demand or supply function doesn't mean that this is how the concept is used in practice. In the real world, *things work the other way round*: economists don't know what the demand curve looks like to begin with, so instead, they estimate the elasticity from data and they use that to get the slope... 

So, you may well ask: why do we give you the exercises where you calculate the elasticity from the slope of the demand curve??
1. The statistical techniques used to estimate elasticities from data are typically taught later in your economics course (2nd year for you guys).
2. The exercises are still very useful in getting you to understand how the two concepts are connected: you need to learn to walk before you can run.